In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from scipy.optimize import fmin_tnc
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
!git clone https://github.com/XiaoxueRan/hku_phys3151_2022

Cloning into 'hku_phys3151_2022'...
remote: Enumerating objects: 326, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 326 (delta 14), reused 1 (delta 1), pack-reused 292
Receiving objects: 100% (326/326), 26.23 MiB | 11.65 MiB/s, done.
Resolving deltas: 100% (164/164), done.


In [3]:
pwd

'/content'

In [4]:
ls

hku_phys3151_2022/  sample_data/


In [5]:
df = pd.read_csv("/content/hku_phys3151_2022/logistic-regression/logistic-regression-example-1.csv")

In [6]:
print(pd.concat([df.head(10), df.tail(10)]))

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
5              5      116             74              0        0  25.6   
6              3       78             50             32       88  31.0   
7             10      115              0              0        0  35.3   
8              2      197             70             45      543  30.5   
9              8      125             96              0        0   0.0   
758            1      106             76              0        0  37.5   
759            6      190             92              0        0  35.5   
760            2       88             

In [7]:
class LogisticRegressionUsingGD:

# Activation function used to map any real value between 0 and 1
    @staticmethod
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

# Computes the weighted sum of inputs Similar to Linear Regression
    @staticmethod
    def net_input(theta, x):
        return np.dot(x, theta)

# Calculates the probability that an instance belongs to a particular class
    def probability(self, theta, x):
        return self.sigmoid(self.net_input(theta, x))

# Computes the cost function for all the training samples
    def cost_function(self, theta, x, y):
        m = x.shape[0]
        total_cost = -(1 / m) * np.sum(
            y * np.log(self.probability(theta, x)) + (1 - y) * np.log(
                1 - self.probability(theta, x)))
        return total_cost

# Computes the gradient of the cost function at the point theta
    def gradient(self, theta, x, y):
        m = x.shape[0]
        return (1 / m) * np.dot(x.T, self.sigmoid(self.net_input(theta, x)) - y)

    def fit(self, x, y, theta):
        opt_weights = fmin_tnc(func=self.cost_function, x0=theta, fprime=self.gradient, args=(x, y.flatten()))
        self.w_ = opt_weights[0]
        return self

    def predict(self, x):
        theta = self.w_[:, np.newaxis]
        return self.probability(theta, x)

    def accuracy(self, x, actual_classes, probab_threshold=0.5):
        predicted_classes = (self.predict(x) >= probab_threshold).astype(int)
        predicted_classes = predicted_classes.flatten()
        accuracy = np.mean(predicted_classes == actual_classes)
        return accuracy * 100

In [8]:
data = df

In [9]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [10]:
X = np.c_[np.ones((X.shape[0], 1)), X]
y = y.values.reshape((len(y),1))
theta = np.zeros((X.shape[1], 1))

In [11]:
model1 = LogisticRegressionUsingGD()
model1.fit(X, y, theta)
accuracy = model1.accuracy(X, y.flatten())
parameters = model1.w_
print("the accuracy of the model is {}".format(accuracy))
print("the model parameters got by gradient descent:")
print(parameters)

the accuracy of the model is 78.25520833333334
the model parameters got by gradient descent:
[-8.33950666e+00  1.22992795e-01  3.49871049e-02 -1.33847683e-02
  6.20495143e-04 -1.17722550e-03  8.89250124e-02  9.37136526e-01
  1.46876967e-02]


In [12]:
model = LogisticRegression()
model.fit(X, y)
parameters = model.coef_
predicted_classes = model.predict(X)
accuracy = accuracy_score(y.flatten(),predicted_classes)
print("the accuracy score using scikit-learn is {}".format(accuracy))
print("the model parameters from scikit- learn:")
print(parameters)

the accuracy score using scikit-learn is 0.77734375
the model parameters from scikit- learn:
[[-3.27488496e+00  1.21062313e-01  3.30765755e-02 -1.29519128e-02
   4.09864825e-03 -1.39238346e-03  7.27109761e-02  1.01564921e+00
   1.27580804e-02]]


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
X1=[[1,6,150,72,36,170,42,1,51],[1,0,85,70,24,200,25,0.2,25]]
print('the prediction of the first method:',model1.predict(X1))

the prediction of the first method: [[0.87281508]
 [0.02308211]]


In [14]:
print('the prediction of the first method:',model.predict(X1))

the prediction of the first method: [1 0]
